In [1]:
import sys, os, glob, shutil
sys.path.append('../')

from MRIsegm.utils import get_slices, get_rois, mask_slices
from PIL import Image


In [2]:
src = '/Users/giuseppefilitto/Pazienti_anonym_sorted'

patients = os.listdir(src)
if '.DS_Store' in patients:
    patients.remove('.DS_Store')

bad_patients = [ 'BO9', 'BO17', 'BO28', 'BO36', 'BO37', 'BO39', 'BO40', 'BO54', 'BO72', 'BO77', 'BO86']

good_patients = list(set(patients) - set(bad_patients))

# Removing because of special folders
special_patients = ['BO38']

good_patients = [x for x in good_patients if x not in special_patients]
print("Number of good patients:",len(good_patients))

Number of good patients: 36


In [3]:
for z, patient in enumerate(good_patients):

    #! slices
    folder = 'T2'
    slices_path = os.path.join(src, patient, folder)

    if not os.path.isdir(slices_path):
        slices_path =  slices_path + "AX"

        if not os.path.isdir(slices_path):
            slices_path = os.path.join(src, patient, 'T25mm')

            if not os.path.isdir(slices_path):
                slices_path = os.path.join(src, patient, 't2DEF')

    if z == 0:
        if len(good_patients) > 1:
            print(f'patient: {patient}', end=', ', flush=True )
        else:
            print(f'patient: {patient}', end='.', flush=True )

    elif z == len(good_patients) - 1 and len(good_patients) > 1:
        print(f'{patient}', end='.', flush=True )
    else:
        print(f'{patient}', end=', ', flush=True )


    dcm_list = sorted(glob.glob(slices_path + '/*.dcm'), key=lambda x: int(os.path.split(x)[1].split('.')[0]))

    #! SLICES

    slices = get_slices(dir_path=slices_path, uint8=True)  

    #! ROIS

    roi_folder = 'T2ROI'
    roi_path = os.path.join(src, patient, roi_folder)
    roi = get_rois(roi_path=roi_path)
    
    slices_of_masks = mask_slices(slices=slices, rois=roi)
    
    positions = [roi[j].get('position') - 1 for j, _ in enumerate(roi)]
    positions = set(positions)

    for i, l in enumerate(positions):

        source = dcm_list[l]
        destination_folder = '../data/imgs'
        filename = patient + '_slice_' + str(l) + '.dcm'

        dest = os.path.join(destination_folder, filename)
        
        if os.path.isfile(dest):
            os.remove(dest)
        shutil.copy(source, dest)

        img = slices[l, ...]
        img = Image.fromarray(img)
        filename = patient + '_slice_' + str(l) + '.png'
        dst = '../data/imgs_frames'
        output =  os.path.join(dst, filename)

        if os.path.isfile(output):
            os.remove(output)
        
        img.save(output)

        mask = slices_of_masks[l, ...]
        mask = Image.fromarray(mask)
        filename = patient + '_label_' + str(l) + '.png'
        dst = '../data/labels'
        output =  os.path.join(dst, filename)

        if os.path.isfile(output):
            os.remove(output)

        mask.save(output)
    
    #! saving no tumor images and labels (black)

        """ if z > 18:
            src_not = dcm_list[0]
            destination_folder = '../data/imgs'
            filename_not = patient + '_slice_' + 'no_tumor' + '.dcm'

            dest_not = os.path.join(destination_folder, filename_not)
            
            if os.path.isfile(dest_not):
                os.remove(dest_not)

            shutil.copy(src_not, dest_not)


            img_not = slices[0, ...]
            img_not = Image.fromarray(img_not)
            filename_not = patient + '_slice_' + 'no_tumor' + '.png'
            dst = '../data/imgs_frames'
            output_not =  os.path.join(dst, filename_not)

            if os.path.isfile(output_not):
                os.remove(output_not)
            
            img_not.save(output_not)

            mask_not = slices_of_masks[0, ...]
            mask_not = Image.fromarray(mask_not)
            filename_not = patient + '_label_' + 'no_tumor' + '.png'
            dst = '../data/labels'
            output_not =  os.path.join(dst, filename_not)
            
            if os.path.isfile(output_not):
                os.remove(output_not)

            mask_not.save(output_not) """

patient: BO26, BO45, BO68, BO74, BO82, BO2, BO56, BO52, BO43, BO60, BO76, BO29, BO64, BO49, BO78, BO33, BO44, BO85, BO48, BO1, BO75, BO35, BO50, BO61, BO71, BO66, BO31, BO42, BO63, BO32, BO16, BO18, BO90, BO47, BO51, BO11.

In [4]:
for z, patient in enumerate(special_patients):
    
    folders = ['T2AXAlta', 'T2AXBassa']
    roi_folders = ['T2ROIalta', 'T2ROIbassa']

    print(f'patient: {patient}', end=' ', flush=True )
        
    for k, _ in enumerate(folders):

        slices_path = os.path.join(src, patient, folders[k])

        if k == len(folders) - 1:
            print(f'{folders[k]}', end='.', flush=True )
        else:
            print(f'folder: {folders[k]}', end=', ', flush=True )

        dcm_list = sorted(glob.glob(slices_path + '/*.dcm'), key=lambda x: int(os.path.split(x)[1].split('.')[0]))

        #! SLICES

        slices = get_slices(dir_path=slices_path, uint8=True) 

        #! ROIS
        roi_path = os.path.join(src, patient, roi_folders[k])
        roi = get_rois(roi_path=roi_path)
        slices_of_masks = mask_slices(slices=slices, rois=roi)
        positions = [roi[j].get('position') - 1 for j, _ in enumerate(roi)]
        positions = set(positions)
     
        for i, l in enumerate(positions):

            source = dcm_list[l]
            destination_folder = '../data/imgs'
            filename = patient + str(folders[k]) + '_slice_' + str(l) + '.dcm'

            dest = os.path.join(destination_folder, filename)
        
            if os.path.isfile(dest):
                os.remove(dest)
            shutil.copy(source, dest)


            img = slices[l, ...]
            img = Image.fromarray(img)
            filename = patient + str(folders[k]) + '_slice_' + str(l) + '.png'
            dst = '../data/imgs_frames'
            output =  os.path.join(dst, filename)

            if os.path.isfile(output):
                os.remove(output)
            
            img.save(output)

            mask = slices_of_masks[l, ...]
            mask = Image.fromarray(mask)
            filename = patient + str(folders[k]) + '_label_' + str(l) + '.png'
            dst = '../data/labels'
            output =  os.path.join(dst, filename)
            if os.path.isfile(output):
                os.remove(output)

            mask.save(output)    

        #! saving no tumor images and labels (black)

        src_not = dcm_list[0]
        destination_folder = '../data/imgs'
        filename_not = patient + str(folders[k]) + '_slice_' + 'no_tumor' + '.dcm'

        dest_not = os.path.join(destination_folder, filename_not)
        
        if os.path.isfile(dest_not):
            os.remove(dest_not)

        shutil.copy(src_not, dest_not)


        img_not = slices[0, ...]
        img_not = Image.fromarray(img_not)
        filename_not = patient + str(folders[k]) + '_slice_' + 'no_tumor' + '.png'
        dst = '../data/imgs_frames'
        output_not =  os.path.join(dst, filename_not)

        if os.path.isfile(output_not):
            os.remove(output_not)
        
        img_not.save(output_not)

        mask_not = slices_of_masks[0, ...]
        mask_not = Image.fromarray(mask_not)
        filename_not = patient + str(folders[k])+ '_label_' + 'no_tumor' + '.png'
        dst = '../data/labels'
        output_not =  os.path.join(dst, filename_not)
        
        if os.path.isfile(output_not):
            os.remove(output_not)

        mask_not.save(output_not) 

patient: BO38 folder: T2AXAlta, T2AXBassa.

In [5]:
img_path = '../data/imgs'
imgs = glob.glob(img_path + '/*.dcm')

print(f'Number of images (.dcm) = {len(imgs)}')

labels_path = '../data/labels'
labels = glob.glob(labels_path + '/*.png')

print(f'Number of labels (.png) = {len(labels)}')

frames_path = '../data/imgs_frames'
frames = glob.glob(frames_path + '/*.png')

print(f'Number of frames (.png) = {len(frames)}')

Number of images (.dcm) = 488
Number of labels (.png) = 488
Number of frames (.png) = 488


In [6]:
 #! OPTIONAL

# from sklearn.model_selection import train_test_split

# X = imgs
# y = labels

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

# print(f'number of imgs before splitting: {len(X)}')
# print(f'number of labels before splitting: {len(y)}')

# print(f'number of training imgs : {len(X_train)}')
# print(f'number of test imgs : {len(X_test)}')

# print(f'number of training labels : {len(y_train)}')
# print(f'number of test labels : {len(y_test)}')



In [7]:
# img_test_path = '../data/imgs_test'
# labels_test_path = '../data/labels_test'
# img_frames_path = '../data/imgs_frames_test'



#for i, j in enumerate(X_test):

#    dir, file = os.path.split(X_test[i])
#    dest = os.path.join(img_test_path, file)

#    if os.path.isfile(dest):
#        os.remove(dest)

#    shutil.move(X_test[i], dest)

#    label_file = file.replace('_slice_','_label_').replace('.dcm','.png')
#    label = os.path.join('../data/labels', label_file)

#    label_dest = os.path.join(labels_test_path, label_file)

#    if os.path.isfile(label_dest):
#        os.remove(label_dest)

#    shutil.move(label, label_dest)

#    frames_file = file.replace('.dcm','.png')
#    frames = os.path.join('../data/imgs_frames', frames_file)

#    frames_dest = os.path.join(img_frames_path, frames_file)

#    if os.path.isfile(frames_dest):
#        os.remove(frames_dest)

#    shutil.move(frames, frames_dest)    

In [8]:
# img_path = '../data/imgs'
# imgs = glob.glob(img_path + '/*.dcm')

# print(f'Number of images (.dcm) after splitting = {len(imgs)}')

# labels_path = '../data/labels'
# labels = glob.glob(labels_path + '/*.png')

# print(f'Number of labels (.png) after slpitting = {len(labels)}')

# frames_path = '../data/imgs_frames'
# frames = glob.glob(frames_path + '/*.png')

# print(f'Number of frames (.png) after splitting = {len(frames)}')